<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=45818d09673b8e8a5e6321df1549903d1db2855ce9cedd96a80762f4784ea8f8
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-05 14:44:37
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.41 C
-------------------
Today PnL: -2.91 L (-2.02%)
Current PnL: -27.20 L (-17.57%)
CY Booked + Current PnL: -12.91 L (-8.34%)
-------------------
Total profit:  1.41 L
Total loss:  -28.61 L
-------------------
Total Booked + Current PnL: 14.07 L (11.01%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.13%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 91.83 L (65.09%)
Deployed:  1.28 C
Current:  1.41 C
CAGR/XIRR %: 5.59%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.12,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.62,10.15,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-4.09,-12.41,14.19,0.02,21695.0,-21667.0,152892.0,133.26,53.0,M-SC,15.37,234.0,-1.00,1.12,32.99,OX40N,NTT,PAINTS
43,ITC,452.00,0.51,-1.02,11.58,10.44,22939.0,-2044.0,198094.0,-40.74,49.0,X-LC,1.56,5.0,-0.09,1.46,4.49,X40,NTT,FMCG
50,MASFIN,398.61,0.00,-2.01,24.55,22.05,23572.0,-1965.0,96015.0,-15.54,60.0,H-SC,7.01,164.0,-0.08,0.71,39.56,XR,ATH,FINANCE
35,ICICIGI,2252.93,0.09,5.32,14.12,20.19,25923.0,9271.0,183591.0,-17.43,46.0,X-MC,6.60,68.0,0.36,1.35,21.24,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,838.00,4.37,-44.62,214.45,74.15,194313.0,-72995.0,90610.0,-262.82,42.0,H-SC,20.02,149.0,-0.38,0.67,16.15,XR,NTT,CHEMICALS
40,INDUSINDBK,1800.00,2.73,-35.77,106.90,32.89,53012.0,-27616.0,49590.0,-712.22,66.0,L-MC,6.24,259.0,-0.52,0.36,36.57,XR,NTT,BANKS
32,HCLTECH,1922.01,2.58,9.21,14.21,24.73,37545.0,22291.0,264215.0,14.03,71.0,X-LC,5.26,7.0,0.59,1.94,25.85,X40,ATH,IT
36,ICICIPRULI,790.00,2.11,3.47,26.57,30.97,48590.0,6138.0,182876.0,-20.92,56.0,X-MC,2.27,75.0,0.13,1.34,16.47,X40,ATH,INSURANCE
60,REPCOHOME,880.00,1.81,-12.12,116.00,89.82,292057.0,-34732.0,251773.0,-56.04,47.0,H-SC,8.26,160.0,-0.12,1.85,33.21,XY24,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.04,-79.81,518.04,24.79,263439.0,-201003.0,50853.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
44,JCHAC,2282.00,-7.38,-36.44,57.33,-0.01,46568.0,-46577.0,81228.0,16530.43,6.0,M-SC,1.08,233.0,-1.00,0.60,0.00,OX40N,NTT,AC
48,KPIGREEN,730.66,-4.95,-14.64,72.16,46.95,77174.0,-18348.0,106949.0,-21.09,34.0,H-SC,3.06,154.0,-0.24,0.79,33.98,MH,ATH,POWER
33,HINDUNILVR,2922.00,-4.92,-8.93,24.82,13.67,58104.0,-22963.0,234100.0,-15.78,32.0,X-LC,3.31,14.0,-0.40,1.72,10.52,XY25,NTT,FMCG
10,BANDHANBNK,400.00,-4.65,-32.67,185.80,92.42,348460.0,-91013.0,187546.0,32.18,25.0,H-SC,6.79,170.0,-0.26,1.38,8.39,XY24,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.85,-6.84,123.19,107.91,169433.0,-10106.0,137538.0,-24.20,30.0,M-SC,11.24,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-4.09,-12.41,14.19,0.02,21695.0,-21667.0,152892.0,133.26,53.0,M-SC,15.37,234.0,-1.00,1.12,32.99,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.12,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.75,253.0,-0.99,0.62,10.15,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.51,-24.30,49.89,13.47,101849.0,-65520.0,204147.0,-68.90,30.0,H-SC,2.40,158.0,-0.64,1.50,3.80,XY24,NTT,PAINTS
72,TATAELXSI,9161.00,0.71,-24.89,75.78,32.03,75036.0,-32817.0,99018.0,-18.72,45.0,H-SC,7.89,148.0,-0.44,0.73,10.91,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.97,6.4,61.9,72.27,111604.0,10849.0,180297.0,-8.8,72.0,M-LC,2.95,99.0,0.1,1.32,12.97,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.97,6.40,61.90,72.27,111604.0,10849.0,180297.0,-8.80,72.0,M-LC,2.95,99.0,0.10,1.32,12.97,XR,NTT,IT
38,INDIAMART,4810.62,-1.43,-1.33,109.50,106.72,133260.0,-1637.0,121699.0,-51.20,36.0,H-SC,1.55,139.0,-0.01,0.89,21.74,AR,ATH,MISC
37,IEX,219.00,-2.71,-1.70,51.19,48.63,100620.0,-3393.0,196561.0,-34.58,56.0,H-SC,14.77,136.0,-0.03,1.44,9.66,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.35,-1.75,38.05,35.63,78347.0,-3668.0,205906.0,-15.41,40.0,H-MC,3.42,119.0,-0.05,1.51,14.63,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.85,-6.84,123.19,107.91,169433.0,-10106.0,137538.0,-24.20,30.0,M-SC,11.24,216.0,-0.06,1.01,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.04,-79.81,518.04,24.79,263439.0,-201003.0,50853.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-1.34,-21.75,89.71,48.45,140754.0,-43607.0,156899.0,-27.39,12.0,X-MC,9.44,64.0,-0.31,1.15,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.26,-37.75,117.18,35.20,93865.0,-48567.0,80103.0,4.98,23.0,X-SC,14.41,92.0,-0.52,0.59,0.04,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.13,-11.89,37.99,21.58,46910.0,-16659.0,123480.0,-55.55,25.0,X-MC,5.75,56.0,-0.36,0.91,9.32,X40N,ATH,IT
54,PGHH,17907.65,-0.47,-6.04,42.36,33.76,79927.0,-12135.0,188685.0,-33.72,28.0,X-MC,4.69,57.0,-0.15,1.39,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-2.18,-24.35,117.00,64.15,210600.0,-57951.0,180000.0,-55.88,38.0,X-SC,1.48,82.0,-0.28,1.32,0.97,XY24,BTT,CEMENT
43,ITC,452.0,0.51,-1.02,11.58,10.44,22939.0,-2044.0,198094.0,-40.74,49.0,X-LC,1.56,5.0,-0.09,1.46,4.49,X40,NTT,FMCG
8,AWL,485.0,1.39,-20.56,93.23,53.50,223477.0,-62046.0,239705.0,-61.59,33.0,X-MC,1.58,58.0,-0.28,1.76,6.47,XY24,NTT,FMCG
15,CAMS,4762.0,-80.04,-79.81,518.04,24.79,263439.0,-201003.0,50853.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
36,ICICIPRULI,790.0,2.11,3.47,26.57,30.97,48590.0,6138.0,182876.0,-20.92,56.0,X-MC,2.27,75.0,0.13,1.34,16.47,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,CAMS,4762.00,-80.04,-79.81,518.04,24.79,263439.0,-201003.0,50853.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
9,BAJAJHFL,181.50,-1.34,-21.75,89.71,48.45,140754.0,-43607.0,156899.0,-27.39,12.0,X-MC,9.44,64.0,-0.31,1.15,0.00,X40N,ATH,FINANCE
54,PGHH,17907.65,-0.47,-6.04,42.36,33.76,79927.0,-12135.0,188685.0,-33.72,28.0,X-MC,4.69,57.0,-0.15,1.39,0.00,X40,ATH,FMCG
55,QUESS,424.00,-2.46,-31.84,109.57,42.84,48548.0,-20698.0,44308.0,-55.37,34.0,X-SC,37.36,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
11,BATAINDIA,2096.00,-1.26,-37.75,117.18,35.20,93865.0,-48567.0,80103.0,4.98,23.0,X-SC,14.41,92.0,-0.52,0.59,0.04,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.0,-2.46,-31.84,109.57,42.84,48548.0,-20698.0,44308.0,-55.37,34.0,X-SC,37.36,83.0,-0.43,0.33,0.00,XY24,NTT,MISC
15,CAMS,4762.0,-80.04,-79.81,518.04,24.79,263439.0,-201003.0,50853.0,-80.65,7.0,X-SC,1.59,86.0,-0.76,0.37,0.00,X40N,NTT,MISC
59,RELAXO,1176.0,-0.80,-47.82,191.88,52.31,145340.0,-69415.0,75745.0,-44.44,33.0,X-SC,5.40,91.0,-0.48,0.56,1.39,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.0,-1.26,-37.75,117.18,35.20,93865.0,-48567.0,80103.0,4.98,23.0,X-SC,14.41,92.0,-0.52,0.59,0.04,X40,NTT,FOOTWEAR
51,MEDANTA,1486.0,-4.45,-3.27,28.27,24.08,34716.0,-4149.0,122801.0,-11.10,32.0,X-SC,6.03,89.0,-0.12,0.90,15.71,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,1.76,-12.08,35.66,19.28,108469.0,-41781.0,304175.0,-23.30,70.0,X-LC,6.53,1.0,-0.39,2.23,12.45,X40,ATH,IT
41,INFY,2275.00,1.01,9.85,40.98,54.87,140856.0,30830.0,343718.0,-14.57,70.0,X-LC,3.25,2.0,0.22,2.52,18.95,X40,BTT,IT
75,TMPV,600.00,-0.62,-16.05,69.37,42.18,160962.0,-44376.0,232034.0,-24.30,30.0,X-LC,3.96,3.0,-0.28,1.70,0.51,XY24,NTT,AUTO
81,VBL,671.64,0.18,-3.37,40.41,35.67,123326.0,-10655.0,305187.0,-14.83,59.0,X-LC,5.31,4.0,-0.09,2.24,10.01,X40N,ATH,FMCG
43,ITC,452.00,0.51,-1.02,11.58,10.44,22939.0,-2044.0,198094.0,-40.74,49.0,X-LC,1.56,5.0,-0.09,1.46,4.49,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.73,-35.77,106.90,32.89,53012.0,-27616.0,49590.0,-712.22,66.0,L-MC,6.24,259.0,-0.52,0.36,36.57,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.55,-17.24,119.52,81.67,92793.0,-16172.0,77638.0,6977.78,38.0,L-SC,16.27,271.0,-0.17,0.57,50.35,XR,NTT,CERAMICS
50,MASFIN,398.61,0.00,-2.01,24.55,22.05,23572.0,-1965.0,96015.0,-15.54,60.0,H-SC,7.01,164.0,-0.08,0.71,39.56,XR,ATH,FINANCE
13,BSOFT,831.70,-0.83,-21.63,92.39,50.79,101847.0,-30417.0,110236.0,0.50,72.0,H-SC,5.21,151.0,-0.30,0.81,28.35,XR,ATH,IT
39,INDIGOPNTS,1408.00,-4.09,-12.41,14.19,0.02,21695.0,-21667.0,152892.0,133.26,53.0,M-SC,15.37,234.0,-1.00,1.12,32.99,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-4.09,-12.41,14.19,0.02,21695.0,-21667.0,152892.0,133.26,53.0,M-SC,15.37,234.0,-1.00,1.12,32.99,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.26,3.63,16.71,20.95,43597.0,9134.0,260902.0,2.60,80.0,X-LC,18.47,31.0,0.21,1.92,40.56,X40,ATH,PAINTS
13,BSOFT,831.70,-0.83,-21.63,92.39,50.79,101847.0,-30417.0,110236.0,0.50,72.0,H-SC,5.21,151.0,-0.30,0.81,28.35,XR,ATH,IT
40,INDUSINDBK,1800.00,2.73,-35.77,106.90,32.89,53012.0,-27616.0,49590.0,-712.22,66.0,L-MC,6.24,259.0,-0.52,0.36,36.57,XR,NTT,BANKS
32,HCLTECH,1922.01,2.58,9.21,14.21,24.73,37545.0,22291.0,264215.0,14.03,71.0,X-LC,5.26,7.0,0.59,1.94,25.85,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.78
1,25,45.70
2,50,76.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.04
MC    30.16
LC    24.79
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.31
X40      22.44
X40N     10.86
XR       10.06
AR        9.39
XY25      9.36
OX40N     7.76
SR        1.02
MH        0.79
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.56
X-MC    23.43
X-LC    21.22
M-SC    12.15
X-SC     6.93
H-MC     4.94
M-MC     1.43
L-SC     1.40
M-LC     1.32
H-LC     1.22
L-LC     1.03
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.39,-6.33,40.72
IT,12.80,-15.56,78.03
FINANCE,9.64,-18.23,68.19
ELECTRICAL,6.26,-10.08,50.40
PAINTS,6.19,-9.56,25.84
MISC,6.14,-55.26,118.87
INSURANCE,4.52,-2.65,38.82
PHARMA,4.07,-0.67,32.89
AUTO,3.44,-22.46,74.96


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3181546.0,21
XR,1292292.0,13
AR,1292083.0,10
X40,983494.0,14
X40N,969906.0,9
OX40N,737434.0,10
XY25,367323.0,6
SR,281605.0,2
MH,77174.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3636752.0,25
M-SC,1406039.0,15
X-MC,1291893.0,16
X-SC,977227.0,8
X-LC,872258.0,11
H-MC,405583.0,3
L-SC,270561.0,3
M-LC,111604.0,1
H-LC,67825.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1278134.0      6
           AR         888002.0      5
M-SC       XY24       798474.0      6
H-SC       XR         774169.0      7
X-SC       X40N       503202.0      3
X-MC       X40        449927.0      7
           XY24       383553.0      3
X-LC       X40        353406.0      5
H-SC       OX40N      337668.0      4
M-SC       OX40N      311469.0      5
X-SC       XY24       293864.0      3
H-SC       SR         281605.0      2
X-MC       X40N       271018.0      4
X-LC       XY24       233341.0      2
H-MC       AR         211403.0      2
X-LC       X40N       195686.0      2
H-MC       XY24       194180.0      1
X-MC       XY25       187395.0      2
L-SC       XR         182264.0      2
X-SC       X40        180161.0      2
M-SC       XR         171243.0      2
           AR         124853.0      2
M-LC       XR         111604.0      1
X-LC       XY25        89825.0      2
L-SC       OX40N       88297.0      1
H-SC       MH          77174.0      1
H-LC       AR          67825.0      1
L-MC       XR          53012.0      1
M-MC       XY25        51202.0      1
L-LC       XY25        38901.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
